In [1]:
!pip install faiss-cpu langchain-huggingface langchain_community sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [ ]:
!pip install transformers  

In [ ]:
import torch, gc 
gc.collect()  
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from charset_normalizer import from_path
import pandas as pd 
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS 
from langchain.chains import RetrievalQA 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

2025-08-22 02:57:25.362462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755831445.549360      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755831445.610565      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
csv_path = "/kaggle/input/q-ands-a/codebasics_faqs.csv"

# 1) Detect encoding
enc = (from_path(csv_path).best() or from_path(csv_path).first()).encoding or "latin1"

# 2) Read CSV (handle both new/old pandas)
read_kwargs = {"encoding": enc}
try: 
    df = pd.read_csv(csv_path, on_bad_lines="skip", **read_kwargs)
except TypeError:
    # for older pandas
    df = pd.read_csv(csv_path, error_bad_lines=False, **read_kwargs) 

# 3) Turn into Documents
try:  
    from langchain_community.document_loaders import DataFrameLoader
except ImportError:
    # Fallback if langchain_community isn't available
    from langchain.document_loaders import DataFrameLoader 

loader = DataFrameLoader(df, page_content_column="prompt")  # <- use your text column
docs = loader.load()

docs[:2]  # quick peek 

[Document(metadata={'response': 'Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.'}, page_content='I have never done programming in my life. Can I take this bootcamp?'),
 Document(metadata={'response': 'Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.'}, page_content='Why should I trust Codebasics?')]

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
vectordb = FAISS.from_documents(documents=docs, embedding=embeddings)    

/tmp/ipykernel_36/368142363.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
retiriver = vectordb.as_retriever()
# rdocs = retiriver.get_relevant_documents("How long the course is valid")  
# rdocs  

In [ ]:
from langchain.prompts import PromptTemplate 

prompt_template = """Given the following context and a question, generate an answer based on this contextonly.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If th answer is not found in the context, kindly state "I Don't Know." Don't try to make up an answer.

CONTEXT: {context}
QUESTION: {question} """
  

PROMPT = PromptTemplate(template = prompt_template, input_variable=["context", "question"])

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B-Instruct-2507") 

pipe = pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    device=0  # or -1 for CPU
)
llm = HuggingFacePipeline(pipeline=pipe) 

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_36/1240307799.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [10]:
chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retiriver, input_key = "query", return_source_documents = True, chain_type_kwargs={"prompt": PROMPT})

In [ ]:
vectordb_filepath = "faiss_index"  
def create_vectordb():
    loader = DataFrameLoader(df, page_content_column="prompt") 
    docs = loader.load()
    vectordb = FAISS.from_documents(documents=docs, embedding=embeddings)
    vectordb.save_local(vectordb_filepath) 
 
if __name__ == "__main__":
    create_vectordb()

In [ ]:
def get_qachain():
    vectordb = FAISS.load_local("/kaggle/working/faiss_index", embeddings, allow_dangerous_deserialization=True)
    retiriver = vectordb.as_retriever(score_threshold=0.7)
    prompt_template = """"
You are a Q&A assistant. 
Given the CONTEXT and QUESTION, extract the most relevant part of the RESPONSE that directly answers the QUESTION. 

- Only return the sentence(s) that answer the QUESTION.   
- If the CONTEXT does not contain the answer, reply with "I Don't Know."  
- Do not include unrelated text.  

CONTEXT:
{context}

QUESTION:
{question}

Final Answer:"""
    PROMPT = PromptTemplate(template = prompt_template, input_variable=["context", "question"])
    chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retiriver, input_key = "query", return_source_documents = True, chain_type_kwargs={"prompt": PROMPT})
    return chain

if __name__=="__main__":
    chain = get_qachain()
    response = chain("Do you guys provide internship?")  
    print(response["result"])

/tmp/ipykernel_36/1893141067.py:25: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain("Do you guys provide internship?")


"
You are a Q&A assistant. 
Given the CONTEXT and QUESTION, extract the most relevant part of the RESPONSE that directly answers the QUESTION. 

- Only return the sentence(s) that answer the QUESTION.  
- If the CONTEXT does not contain the answer, reply with "I Don't Know."  
- Do not include unrelated text.  

CONTEXT:
Do you provide any virtual internship?

Do you provide any job assistance?

Will this course guarantee me a job?

Can I add this course to my resume?

QUESTION:
Do you guys provide internship?

Final Answer: Yes, we provide virtual internships.
"

CONTEXT:
Do you provide any virtual internship?

Do you provide any job assistance?

Will this course guarantee me a job?

Can I add this course to my resume?

QUESTION:
Do you provide internship?

Final Answer: Yes, we provide virtual internships.
" CONTEXT:
Do you provide any virtual internship?

Do you provide any job assistance?

Will this course guarantee me a job?

Can I add this course to my resume?

QUESTION:
Do you p

In [ ]:
!ngrok config add-authtoken 1d0hmrzQ9X0oTgUoCiUFBrZjuAW_3bFsz96evrBkAVuYfNwBY 

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [15]:
!pip install pyngrok

In [ ]:
import nest_asyncio
from pyngrok import ngrok 
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn


nest_asyncio.apply()


app = FastAPI()


class Question(BaseModel): 
    query: str

@app.post("/ask")
def ask(question: Question):
    chain = get_qachain()
    return {"answer": response["result"]}
    
    if "Final Answer:" in result:
        final_answer = result.split("Final Answer:")[-1].strip()
    else:
        final_answer = result.strip()

    return {"answer": final_answer}


public_url = ngrok.connect(8000)  # opens tunnel
print("🚀 Public URL:", public_url)

uvicorn.run(app, host="0.0.0.0", port=8000) 

🚀 Public URL: NgrokTunnel: "https://ba41ed0e52c0.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     117.99.4.22:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     117.99.4.22:0 - "POST /ask HTTP/1.1" 200 OK
